In [1]:
import paddle
paddle.set_device('cpu')

CPUPlace

# steps
1. load model
2. load preprocess_ops
3. extract_feat
4. load Gallery
5. vec search

In [2]:
from python.preprocess import create_operators
from python.postprocess import build_postprocess
from utils.predictor import Predictor
from utils.get_image_list import get_image_list
from utils import config
import numpy as np
import cv2
import os
# from torch.utils.data import Dataset, DataLoader

In [3]:
class RecPredictor(Predictor):
    def __init__(self, config):
        super().__init__(config["Global"],
                         config["Global"]["rec_inference_model_dir"])
        self.preprocess_ops = create_operators(config["RecPreProcess"][
            "transform_ops"])
        self.postprocess = build_postprocess(config["RecPostProcess"])
        self.benchmark = config["Global"].get("benchmark", False)
    
    def predict(self, images, feat_norm=True):
        input_names = self.predictor.get_input_names()
        input_tensor = self.predictor.get_input_handle(input_names[0])

        output_names = self.predictor.get_output_names()
        output_tensor = self.predictor.get_output_handle(output_names[0])
        
        if not isinstance(images, (list, )):
            images = [images]
        
        for idx in range(len(images)):
            for ops in self.preprocess_ops:
                images[idx] = ops(images[idx])
        image = np.array(images)
        input_tensor.copy_from_cpu(image)
        self.predictor.run()
        batch_output = output_tensor.copy_to_cpu()
        
        if feat_norm:
            feas_norm = np.sqrt(
                np.sum(np.square(batch_output), axis=1, keepdims=True))
            batch_output = np.divide(batch_output, feas_norm)
            
        
        if self.postprocess is not None:
            batch_output = self.postprocess(batch_output)
            
        return batch_output

In [4]:
fname = '/home/cloud623b/student/lzq/PaddleClas/deploy/configs/xmg_test.yaml'
config = config.get_config(fname, show=False)
rec_predictor = RecPredictor(config)

In [5]:
def read_imgs(image_list):
    imgs = []
    names = []
    
    for img_path in image_list:
        img = cv2.imread(img_path) # image (H, W, C-BGR)

        img = img[:, :, ::-1] # bgr2rgb
        imgs.append(img)
        img_name = os.path.basename(img_path)
        names.append(img_name)
    return imgs, names
    

In [6]:
import faiss
import pickle
idx_file = '/home/cloud623b/student/lzq/PaddleClas/deploy/xmg_dataset/index/vector.index'
searcher = faiss.read_index(idx_file)
id_map_path = '/home/cloud623b/student/lzq/PaddleClas/deploy/xmg_dataset/index/id_map.pkl'
with open(id_map_path, 'rb') as fd:
    id_map = pickle.load(fd)

2022-04-04 20:13:47 INFO: Loading faiss with AVX2 support.
2022-04-04 20:13:47 INFO: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2022-04-04 20:13:47 INFO: Loading faiss.
2022-04-04 20:13:47 INFO: Successfully loaded faiss.


In [8]:
def inference_by_batch(imgs, predictor, batch_size=32):
    cnt = 0
    batch_imgs = []
    res = []
    for idx, img in enumerate(imgs):
        batch_imgs.append(img)
        cnt += 1
        if cnt % batch_size == 0 or (idx + 1) == len(imgs):
            if len(batch_imgs) == 0:
                continue
            batch_results = predictor.predict(batch_imgs)
            res.append(batch_results)
            batch_imgs = []
    
    return np.concatenate(res, axis=0)

In [13]:
import pandas as pd
id2name_dict = pd.read_csv('/home/cloud623b/student/lzq/PaddleClas/deploy/xmg_dataset/item2id.csv')['物品名称']

In [18]:
def test_all_dir(idxs, id2name_dict, base_dir):
    precision_dict = {}
    for idx in idxs:
        name = id2name_dict[idx]
        path = os.path.join(base_dir, str(idx))
        image_list = get_image_list(path)
        imgs, names = read_imgs(image_list)
        
        rec_results = rec_predictor.predict(imgs)
        return_k = 5
        scores, docs = searcher.search(rec_results, return_k)
        
        cnt = 0
        for doc in docs:
            if id_map[doc[0]].endswith(name):
                cnt += 1
        
        precision_dict[idx] = cnt / len(image_list)
    return precision_dict
    

In [20]:
# image_list = get_image_list(config["Global"]["infer_imgs"])
base_dir = '/home/cloud623b/student/lzq/PaddleClas/deploy/xmg_dataset/test_set/'

In [21]:
res = test_all_dir(range(1, 13), id2name_dict, base_dir)

In [22]:
res

{1: 0.21921182266009853,
 2: 0.1271186440677966,
 3: 0.833729216152019,
 4: 0.4923547400611621,
 5: 0.5864864864864865,
 6: 0.2248995983935743,
 7: 0.22337662337662337,
 8: 0.24043715846994534,
 9: 0.34798534798534797,
 10: 0.21680216802168023,
 11: 0.7139240506329114,
 12: 0.011185682326621925}

In [ ]:
from torch.utils.data.dataloader import DataLoader